# Kompozyt

Wzorzec Kompozyt (ang. Composite) umożliwia reprezentowanie hierarchicznych struktur obiektów, w których zarówno pojedyncze elementy lub i ich złożone grupy mogą być traktowane w jednolity sposób. Kompozyt definiuje wspólny i spójny interfejs dla obiektów prostych (liści drzewa) oraz obiektów złożonych (kontenerów), co pozwala na ich rekurencyjne przetwarzanie. Dzięki temu klient nie musi rozróżniać, czy pracuje z pojedynczym elementem, czy z całą strukturą – może używać ich w ten sam sposób. Kompozyt jest często stosowany w sytuacjach, gdzie system wymaga modelowania drzewiastej struktury, np. plików w systemie, interfejsów graficznych czy hierarchii organizacyjnych.

## Przeznaczenie i zastosowanie

- Ujednolicenie operacji na pojedynczych obiektach i w grupach, umożliwiając traktowanie ich w ten sam sposób.
- Reprezentowanie hierarchii obiektów w postaci drzewa, np. struktury katalogów.
- Zwiększenie elastyczności i skalowalności kodu umożliwiając w tej sposób łatwe rozszerzanie struktury bez zmiany kodu klienta.

<img src="img/Composite_Design_Pattern_UML.jpg">

<img src="img/Composite_UML_class_diagram.svg" width="50%">

## Implementacja

Cel: przesunięcie trójkąta o wybrany wektor. 

Hierarchia klas trójkąta:
- Point: punkt na płaszczyźnie,
- Segment: odcinek składający się z punktów na każdym końcu,
- Triangle: trójkąt składający się z odcinków.

In [ ]:
from abc import ABC, abstractmethod
from typing import Self

Klasa definiująca interfejs każdego z komponentów trójkąta.

In [ ]:
class Component(ABC):
    _parent: Self

    def __init__(self) -> None:
        self._parent = None
        
    @property
    def parent(self) -> Self:
        return self._parent
    
    @parent.setter
    def parent(self, parent: Self) -> None:
        self._parent = parent
        
    def get_parent(self) -> Self:
        return self._parent
        
    def add(self, component: Self) -> None:
        pass
    
    def remove(self, component: Self) -> None:
        pass
    
    def is_composite(self) -> bool:
        return False
    
    @abstractmethod
    def move(self, a: float, b: float) -> None:
        pass

Klasa reprezentująca punkt.

In [ ]:
class Point(Component):
    x: int
    y: int

    def __init__(self, x: int, y: int) -> None:
        super().__init__()
        self.x = x
        self.y = y
        
    def move(self, a: int, b: int) -> None:
        self.x += a
        self.y += b
        
        print("point moved")

Klasa reprezentująca odcinek. Jest to kompozyt - zawiera referencję do liści w postaci odcinków.

In [ ]:
class Segment(Component):
    begin: Point
    end: Point
    children: list

    def __init__(self, begin: Point, end: Point) -> None:
        super().__init__()
        self.begin = begin
        self.end = end
        self.children = []
        self.add(begin)
        self.add(end)
        
    def add(self, component: Point) -> None:
        component.parent = self
        self.children.append(component)
    
    def remove(self, component: Point) -> None:
        component.parent = None
        self.children.remove(component)
    
    def is_composite(self) -> bool:
        return True
    
    def move(self, a: int, b: int) -> None:
        for child in self.children:
            child.move(a, b)
        
        print("segment moved")

Klasa reprezentująca trójkąt. Zawiera referencje do odcinków (i pośrednio do punktów). Wględem hierarchii - również jest to kompozyt.

In [ ]:
class Traingle(Component):
    children: list

    def __init__(self, seg0: Segment, seg1: Segment, seg2: Segment) -> None:
        super().__init__()
        self.children = []
        self.add(seg0)
        self.add(seg1)
        self.add(seg2)
        
    def add(self, component: Segment) -> None:
        component.parent = self
        self.children.append(component)
    
    def remove(self, component: Segment) -> None:
        component.parent = None
        self.children.remove(component)
        
    def is_composite(self) -> bool:
        return True
    
    def move(self, a: int, b: int) -> None:
        for child in self.children:
            child.move(a, b)
        
        print("traingle moved")

### Kod klienta

Utworzenie i przesunięcie punktów

In [ ]:
point_a1 = Point(3, 5)
point_a2 = Point(10, 10)

In [ ]:
point_a1.move(-1, -3)

Utworzenie i przesunięcie odcinków

In [ ]:
point_b1 = Point(-3, 1)
point_b2 = Point(7, 11)
point_c1 = Point(1, 0)
point_c2 = Point(0, 1)

seg1 = Segment(point_a1, point_a2)
seg2 = Segment(point_b1, point_b2)
seg3 = Segment(point_c1, point_c2)

In [ ]:
seg2.move(5, 5)

Utworzenie i przesunięcie trójkąta

In [ ]:
triangle = Traingle(seg1, seg2, seg3)
triangle.move(11.5, 12.25)

## Podsumowanie

Wzorzec Kompozyt umożliwia reprezentowanie hierarchicznych struktur obiektów, w których zarówno pojedyncze elementy lub i ich złożone grupy mogą być traktowane w jednolity sposób. Taki proces rodzi pewne konsekwencje:
- rekurencyjne wykonania metod komponentów,
- reprezentacja obiektów za pomocą struktury drzewa,
- łatwe wykonanie metod zbiorczych, takich jak np. "przesuń wszystkie punkty, które tworzą odcinek i trójkąt",
- ciągła mamy możliwość odwołania się do komponentów składowych,
- dodanie elementów pośrednich bez konieczności przebudowy całego projektu,
- implementacje wielu komponentów mogą być zbyt ogólne: np. klasa `Point` posiada metody `add` i `remove` poprzez dziedziczenie, które nie są wykorzystywane.